진호님이 일단 실행되게끔 만들어 본것

In [40]:
# !pip install konlpy

In [41]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리 해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file( 'test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')



In [42]:
import pandas as pd
#x,y값을 추출후 데이터프레임에 넣는다.
# X 값 추출 피쳐feature, y값 추출target 

df_train = pd.DataFrame({
    'feature':[row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0],
    'target': [ int(row.split('\t')[2]) for row in train_text.split('\n')[1:] if row.count('\t')>0]
})


In [43]:
# df_train.iloc[:,0]
from konlpy.tag import Okt

#문장을 토큰화한것만 따로 저장
# tokenized_data =[]

# for i in df_train.iloc[:,0]:
#     tokenized_data.append(Okt().morphs( i ))

In [44]:
import pickle

#토근화된 데이터를 저장하였음
# with open('tokenized_data.pkl','wb') as f:
#     pickle.dump(tokenized_data,f)

#토큰화된 데이터 불러오기
# with open('tokenized_data.pkl','rb') as f:
#     tokenized_data_loaded = pickle.load(f)

# tokenized_data_loaded

In [46]:
#라벨별로 분리해본것
bad = df_train[df_train.iloc[:,1]==0]
good = df_train[df_train.iloc[:,1]==1]


df_train = df_train[::4]

from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer=TfidfVectorizer(tokenizer=Okt().morphs)
X = vectorizer.fit_transform( df_train.iloc[:,0])
y = df_train.iloc[:,1]


In [83]:
df_train[df_train.iloc[:,1]==0]  #18678 
df_train[df_train.iloc[:,1]==1]  #18822
df_train.shape

(37500, 2)

In [48]:
# 토큰화하고 빈도수(tfidf)에 기반한 벡터표현이 생성된 X를 따로저장
from scipy.sparse import save_npz ,load_npz

#vectorizer.fit_transform한것을 희소행렬로로 저장
# save_npz('vectorized_data.npz', X)

#저장된 희소행렬을 불러오기
# X_loaded = load_npz('vectorized_data.npz')

#이렇게 불러온 X_loaded는 바로 모델학습에 사용가능


In [49]:
import numpy as np
X_train_dense = np.expand_dims( X.toarray(), axis=-1 )
# X_train_dense.shape
# X_loaded.shape[1]
# X_loaded.toarray().shape[1]

In [104]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization , Input, Dropout
# from tensorflow.keras import Input
# 모델 구성deep nueral network
# input_layer = Input(shape=(X_loaded.shape[1],), sparse=True)

model = Sequential([
    # input_layer,
    Input(shape=(X_train_dense.shape[1],)),
    Dense(3, activation='relu'),
    BatchNormalization(),
    Dropout(0.6),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(X.toarray(), y, epochs = 10, batch_size = 500, validation_split=0.2)  #0.8417
model.fit(X.toarray(), y, epochs = 10, batch_size = 20, validation_split=0.3)  #0.8417

Epoch 1/10
1313/1313 [==============================] - 16s 10ms/step - loss: 0.5540 - accuracy: 0.6929 - val_loss: 0.4232 - val_accuracy: 0.8061
Epoch 2/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.4514 - accuracy: 0.7635 - val_loss: 0.3957 - val_accuracy: 0.8207
Epoch 3/10
1313/1313 [==============================] - 12s 9ms/step - loss: 0.3995 - accuracy: 0.7956 - val_loss: 0.3923 - val_accuracy: 0.8203
Epoch 4/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.3699 - accuracy: 0.8093 - val_loss: 0.3990 - val_accuracy: 0.8155
Epoch 5/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.3484 - accuracy: 0.8202 - val_loss: 0.4058 - val_accuracy: 0.8202
Epoch 6/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.3330 - accuracy: 0.8279 - val_loss: 0.4146 - val_accuracy: 0.8172
Epoch 7/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.3185 - accuracy: 0.8337 - val_loss: 0.4246 - val_a

In [107]:
test = ['정말 별로 입니다. 이게뭐니']

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)

# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round( predictions ).astype(int))

1/1 [==============================] - 0s 49ms/step
[[0.01700829]]
예측 결과: [[0]]


In [25]:
# model.save('기본 DNN 적용모델.h5')